# Comprehensive Trading System Analysis

This notebook provides complete analysis of the full trading system including signals, portfolio, and execution.

**Key Features:**
- Signal generation analysis
- Trade execution analysis
- Portfolio performance metrics
- Risk analysis
- Execution cost analysis
- Position and fill analysis

In [1]:
# Parameters will be injected here by papermill
# This cell is tagged with 'parameters' for papermill to recognize it
run_dir = "."
config_name = "config"
symbols = ["SPY"]
timeframe = "5m"

# Analysis parameters
execution_cost_bps = 1.0  # Round-trip execution cost in basis points
analyze_slippage = True
analyze_intraday_patterns = True
market_timezone = "America/New_York"

# Performance thresholds
min_sharpe_ratio = 1.0
max_acceptable_drawdown = 0.20  # 20%
min_win_rate = 0.45

In [2]:
# Parameters
run_dir = "/Users/daws/ADMF-PC/config/bollinger/results/20250703_201201"
config_name = "bollinger"
symbols = ["SPY"]
timeframe = "5m"
global_traces_dir = "/Users/daws/ADMF-PC/traces"
min_strategies_to_analyze = 20
sharpe_threshold = 1.0
correlation_threshold = 0.7
top_n_strategies = 10
ensemble_size = 5
calculate_all_performance = True
performance_limit = 100


## Setup

In [3]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime, time
import pytz
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Convert run_dir to Path
run_dir = Path(run_dir).resolve()
print(f"Analyzing run: {run_dir.name}")
print(f"Full path: {run_dir}")
print(f"Config: {config_name}")
print(f"Symbol(s): {symbols}")
print(f"Timeframe: {timeframe}")

Analyzing run: 20250703_201201
Full path: /Users/daws/ADMF-PC/config/bollinger/results/20250703_201201
Config: bollinger
Symbol(s): ['SPY']
Timeframe: 5m


## Load Metadata and Traces

In [4]:
# Load run metadata
metadata_path = run_dir / 'metadata.json'
if metadata_path.exists():
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    print(f"✅ Run metadata loaded")
    print(f"   Total bars: {metadata.get('total_bars', 'N/A')}")
    print(f"   Total signals: {metadata.get('total_signals', 'N/A')}")
    print(f"   Total orders: {metadata.get('total_orders', 'N/A')}")
    print(f"   Total fills: {metadata.get('total_fills', 'N/A')}")
    print(f"   Total positions: {metadata.get('total_positions', 'N/A')}")
    
    # Get global traces path
    global_traces_path = Path(metadata.get('global_traces_path', '/Users/daws/ADMF-PC/traces'))
    print(f"\n📁 Global traces path: {global_traces_path}")
else:
    print("❌ No metadata.json found")
    metadata = {}
    global_traces_path = Path('/Users/daws/ADMF-PC/traces')

# Check what traces are available in global store
store_path = global_traces_path / 'store'
has_global_signals = store_path.exists() and any(store_path.glob('*.parquet'))

# For backward compatibility, also check run directory
traces_dir = run_dir / 'traces'
has_local_signals = (traces_dir / 'signals').exists() if traces_dir.exists() else False
has_portfolio = (traces_dir / 'portfolio').exists() if traces_dir.exists() else False
has_execution = (traces_dir / 'execution').exists() if traces_dir.exists() else False

print(f"\n📊 Available traces:")
print(f"   Global signals: {'✅' if has_global_signals else '❌'}")
print(f"   Local signals: {'✅' if has_local_signals else '❌'}")
print(f"   Portfolio: {'✅' if has_portfolio else '❌'}")
print(f"   Execution: {'✅' if has_execution else '❌'}")

# Determine trace location
use_global_store = has_global_signals and not has_local_signals
is_full_system = metadata.get('total_orders', 0) > 0 or has_portfolio or has_execution

✅ Run metadata loaded
   Total bars: 16614
   Total signals: 16601
   Total orders: 2066
   Total fills: 2066
   Total positions: 2066

📁 Global traces path: /Users/daws/ADMF-PC/traces

📊 Available traces:
   Global signals: ✅
   Local signals: ❌
   Portfolio: ❌
   Execution: ❌


## Load Market Data

In [5]:
# Load market data
market_data = None
for symbol in symbols:
    try:
        # Try different possible locations
        data_paths = [
            run_dir / f'data/{symbol}_{timeframe}.csv',
            run_dir / f'{symbol}_{timeframe}.csv',
            run_dir.parent / f'data/{symbol}_{timeframe}.csv',
            Path(f'/Users/daws/ADMF-PC/data/{symbol}_{timeframe}.csv')
        ]
        
        for data_path in data_paths:
            if data_path.exists():
                market_data = pd.read_csv(data_path)
                market_data['timestamp'] = pd.to_datetime(market_data['timestamp'])
                market_data = market_data.sort_values('timestamp')
                
                # Add derived fields
                market_data['returns'] = market_data['close'].pct_change()
                market_data['log_returns'] = np.log(market_data['close'] / market_data['close'].shift(1))
                market_data['hour'] = market_data['timestamp'].dt.hour
                market_data['minute'] = market_data['timestamp'].dt.minute
                market_data['day_of_week'] = market_data['timestamp'].dt.dayofweek
                
                print(f"✅ Loaded market data from: {data_path}")
                print(f"   Date range: {market_data['timestamp'].min()} to {market_data['timestamp'].max()}")
                print(f"   Total bars: {len(market_data)}")
                break
        
        if market_data is not None:
            break
            
    except Exception as e:
        print(f"Error loading data for {symbol}: {e}")

if market_data is None:
    print("❌ Could not load market data")

✅ Loaded market data from: /Users/daws/ADMF-PC/data/SPY_5m.csv
   Date range: 2024-03-26 13:30:00+00:00 to 2025-04-02 19:20:00+00:00
   Total bars: 20769


## Signal Analysis

In [6]:
# Load and analyze signals if available
if has_global_signals or has_local_signals:
    print("\n📊 SIGNAL ANALYSIS")
    print("=" * 80)
    
    # Load strategy index
    strategy_index_path = run_dir / 'strategy_index.parquet'
    if strategy_index_path.exists():
        strategy_index = pd.read_parquet(strategy_index_path)
        print(f"Loaded {len(strategy_index)} strategies from run index")
        
        # Show strategy distribution
        by_type = strategy_index['strategy_type'].value_counts()
        print("\nStrategies by type:")
        for stype, count in by_type.items():
            print(f"  {stype}: {count}")
    else:
        print("No strategy index found in run directory")
        strategy_index = pd.DataFrame()
    
    # Analyze signal patterns from global store
    if use_global_store and len(strategy_index) > 0:
        print("\n📊 Analyzing signals from global store...")
        
        # Get trace paths from metadata components
        signal_counts = []
        components = metadata.get('components', {})
        
        for comp_name, comp_data in components.items():
            if comp_data.get('type') == 'strategy' and 'trace_path' in comp_data:
                trace_path = Path(comp_data['trace_path'])
                if trace_path.exists():
                    signals = pd.read_parquet(trace_path)
                    
                    # Count actual signal changes (non-zero values)
                    signal_changes = signals[signals['val'] != 0]
                    
                    signal_counts.append({
                        'strategy_type': comp_data.get('strategy_type'),
                        'strategy_hash': comp_data.get('strategy_hash'),
                        'total_signals': len(signals),
                        'signal_changes': len(signal_changes),
                        'long_signals': (signal_changes['val'] > 0).sum(),
                        'short_signals': (signal_changes['val'] < 0).sum(),
                        'signals_per_1000_bars': len(signal_changes) / (metadata.get('total_bars', 1000) / 1000)
                    })
                    
                    # Show sample signals
                    if len(signal_changes) > 0:
                        print(f"\n  Strategy: {comp_data.get('strategy_type')} ({comp_data.get('strategy_hash', '')[:8]})")
                        print(f"    Signal changes: {len(signal_changes)}")
                        print(f"    First signal: {signal_changes.iloc[0]['ts']} -> {signal_changes.iloc[0]['val']}")
                        print(f"    Last signal: {signal_changes.iloc[-1]['ts']} -> {signal_changes.iloc[-1]['val']}")
        
        if signal_counts:
            signal_df = pd.DataFrame(signal_counts)
            print("\n📊 Signal frequency analysis:")
            print(signal_df.to_string(index=False))
            
            # Check if signals were generated but no trades
            if metadata.get('total_signals', 0) > 0 and metadata.get('total_orders', 0) == 0:
                print("\n⚠️ WARNING: Signals were generated but no orders were created!")
                print("Possible reasons:")
                print("  - Risk constraints (stop loss/take profit) may be too tight")
                print("  - Position sizing returned 0 shares")
                print("  - Intraday constraints prevented trades")
                print("  - Check the execution logs for more details")
        else:
            print("\n⚠️ No signal traces found in global store")
            
    # Analyze from local traces (backward compatibility)
    elif has_local_signals:
        print("\n📊 Analyzing signals from local traces...")
        # Original local trace analysis code here
else:
    print("\n⚠️ No signal traces available")


📊 SIGNAL ANALYSIS
No strategy index found in run directory


## Portfolio Analysis

In [7]:
# Load and analyze portfolio data if available
if has_portfolio:
    print("\n💼 PORTFOLIO ANALYSIS")
    print("=" * 80)
    
    # Load orders
    orders_path = traces_dir / 'portfolio' / 'orders' / 'orders.parquet'
    if orders_path.exists():
        orders = pd.read_parquet(orders_path)
        orders['timestamp'] = pd.to_datetime(orders['timestamp'])
        print(f"Loaded {len(orders)} orders")
        
        # Order analysis
        print("\nOrder Statistics:")
        print(f"  Buy orders: {(orders['side'] == 'buy').sum() if 'side' in orders else 'N/A'}")
        print(f"  Sell orders: {(orders['side'] == 'sell').sum() if 'side' in orders else 'N/A'}")
        if 'order_type' in orders:
            print("\nOrder types:")
            print(orders['order_type'].value_counts())
    else:
        orders = pd.DataFrame()
        print("No orders found")
    
    # Load positions
    positions_open_path = traces_dir / 'portfolio' / 'positions_open' / 'positions_open.parquet'
    positions_close_path = traces_dir / 'portfolio' / 'positions_close' / 'positions_close.parquet'
    
    positions_opened = pd.DataFrame()
    positions_closed = pd.DataFrame()
    
    if positions_open_path.exists():
        positions_opened = pd.read_parquet(positions_open_path)
        positions_opened['timestamp'] = pd.to_datetime(positions_opened['timestamp'])
        print(f"\nLoaded {len(positions_opened)} position opens")
    
    if positions_close_path.exists():
        positions_closed = pd.read_parquet(positions_close_path)
        positions_closed['timestamp'] = pd.to_datetime(positions_closed['timestamp'])
        print(f"Loaded {len(positions_closed)} position closes")
    
    # Analyze positions
    if len(positions_opened) > 0 and len(positions_closed) > 0:
        # Match opens and closes
        trades = []
        for _, close in positions_closed.iterrows():
            # Find matching open
            position_id = close.get('position_id', close.get('metadata', {}).get('position_id'))
            if position_id:
                matching_opens = positions_opened[
                    positions_opened.apply(lambda x: x.get('position_id', x.get('metadata', {}).get('position_id')) == position_id, axis=1)
                ]
                if len(matching_opens) > 0:
                    open_pos = matching_opens.iloc[0]
                    trades.append({
                        'position_id': position_id,
                        'symbol': close.get('symbol', close.get('metadata', {}).get('symbol')),
                        'entry_time': open_pos['timestamp'],
                        'exit_time': close['timestamp'],
                        'entry_price': open_pos.get('price', open_pos.get('metadata', {}).get('entry_price', 0)),
                        'exit_price': close.get('price', close.get('metadata', {}).get('exit_price', 0)),
                        'quantity': close.get('metadata', {}).get('quantity', 0),
                        'pnl': close.get('metadata', {}).get('realized_pnl', 0)
                    })
        
        if trades:
            trades_df = pd.DataFrame(trades)
            trades_df['duration'] = (trades_df['exit_time'] - trades_df['entry_time']).dt.total_seconds() / 60  # minutes
            trades_df['return'] = (trades_df['exit_price'] - trades_df['entry_price']) / trades_df['entry_price']
            
            print("\n📊 Trade Statistics:")
            print(f"  Total trades: {len(trades_df)}")
            print(f"  Win rate: {(trades_df['pnl'] > 0).mean()*100:.1f}%")
            print(f"  Average PnL: ${trades_df['pnl'].mean():.2f}")
            print(f"  Total PnL: ${trades_df['pnl'].sum():.2f}")
            print(f"  Average duration: {trades_df['duration'].mean():.1f} minutes")
            print(f"  Average return: {trades_df['return'].mean()*100:.3f}%")
else:
    print("\n⚠️ No portfolio traces available")
    trades_df = pd.DataFrame()


⚠️ No portfolio traces available


## Execution Analysis

In [8]:
# Load and analyze execution data if available
if has_execution:
    print("\n⚡ EXECUTION ANALYSIS")
    print("=" * 80)
    
    # Load fills
    fills_path = traces_dir / 'execution' / 'fills' / 'fills.parquet'
    if fills_path.exists():
        fills = pd.read_parquet(fills_path)
        fills['timestamp'] = pd.to_datetime(fills['timestamp'])
        print(f"Loaded {len(fills)} fills")
        
        # Extract metadata
        if 'metadata' in fills.columns:
            # Expand metadata column
            fill_details = pd.json_normalize(fills['metadata'])
            fills = pd.concat([fills, fill_details], axis=1)
        
        # Fill analysis
        print("\nFill Statistics:")
        if 'quantity' in fills:
            print(f"  Total volume: {fills['quantity'].sum():,.0f} shares")
            print(f"  Average fill size: {fills['quantity'].mean():.0f} shares")
        
        if 'price' in fills:
            print(f"  Average fill price: ${fills['price'].mean():.2f}")
        
        # Slippage analysis
        if analyze_slippage and 'expected_price' in fills and 'price' in fills:
            fills['slippage'] = (fills['price'] - fills['expected_price']) / fills['expected_price']
            fills['slippage_bps'] = fills['slippage'] * 10000
            
            print("\n💸 Slippage Analysis:")
            print(f"  Average slippage: {fills['slippage_bps'].mean():.1f} bps")
            print(f"  Slippage std dev: {fills['slippage_bps'].std():.1f} bps")
            print(f"  Positive slippage: {(fills['slippage'] > 0).mean()*100:.1f}%")
            print(f"  Total slippage cost: ${(fills['slippage'] * fills['quantity'] * fills['price']).sum():.2f}")
        
        # Execution cost analysis
        if 'commission' in fills:
            print("\n💰 Execution Costs:")
            print(f"  Total commissions: ${fills['commission'].sum():.2f}")
            print(f"  Average commission: ${fills['commission'].mean():.2f}")
            print(f"  Commission as % of volume: {fills['commission'].sum() / (fills['quantity'] * fills['price']).sum() * 100:.3f}%")
    else:
        fills = pd.DataFrame()
        print("No fills found")
else:
    print("\n⚠️ No execution traces available")
    fills = pd.DataFrame()


⚠️ No execution traces available


## Performance Metrics

In [9]:
# Calculate overall performance metrics if we have trades
if len(trades_df) > 0:
    print("\n📈 PERFORMANCE METRICS")
    print("=" * 80)
    
    # Calculate equity curve from trades
    initial_capital = 100000  # Assumed
    equity = initial_capital
    equity_curve = [{'timestamp': trades_df['entry_time'].min(), 'equity': equity}]
    
    for _, trade in trades_df.sort_values('exit_time').iterrows():
        equity += trade['pnl']
        equity_curve.append({'timestamp': trade['exit_time'], 'equity': equity})
    
    equity_df = pd.DataFrame(equity_curve)
    equity_df['returns'] = equity_df['equity'].pct_change()
    
    # Performance metrics
    total_return = (equity_df['equity'].iloc[-1] / initial_capital - 1)
    
    # Sharpe ratio (assuming daily returns)
    daily_returns = trades_df.groupby(trades_df['exit_time'].dt.date)['pnl'].sum() / equity
    if len(daily_returns) > 1 and daily_returns.std() > 0:
        sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
    else:
        sharpe_ratio = 0
    
    # Max drawdown
    cummax = equity_df['equity'].expanding().max()
    drawdown = (equity_df['equity'] / cummax - 1)
    max_drawdown = drawdown.min()
    
    # Win/loss statistics
    winning_trades = trades_df[trades_df['pnl'] > 0]
    losing_trades = trades_df[trades_df['pnl'] <= 0]
    
    print(f"Total Return: {total_return*100:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_drawdown*100:.2f}%")
    print(f"\nTrade Statistics:")
    print(f"  Win Rate: {len(winning_trades)/len(trades_df)*100:.1f}%")
    print(f"  Average Win: ${winning_trades['pnl'].mean():.2f}" if len(winning_trades) > 0 else "  Average Win: N/A")
    print(f"  Average Loss: ${losing_trades['pnl'].mean():.2f}" if len(losing_trades) > 0 else "  Average Loss: N/A")
    print(f"  Profit Factor: {winning_trades['pnl'].sum() / abs(losing_trades['pnl'].sum()):.2f}" if len(losing_trades) > 0 and losing_trades['pnl'].sum() != 0 else "  Profit Factor: N/A")
    
    # Performance vs thresholds
    print(f"\n🎯 Performance vs Thresholds:")
    print(f"  Sharpe Ratio: {sharpe_ratio:.2f} {'✅' if sharpe_ratio >= min_sharpe_ratio else '❌'} (min: {min_sharpe_ratio})")
    print(f"  Max Drawdown: {abs(max_drawdown)*100:.1f}% {'✅' if abs(max_drawdown) <= max_acceptable_drawdown else '❌'} (max: {max_acceptable_drawdown*100:.0f}%)")
    print(f"  Win Rate: {len(winning_trades)/len(trades_df)*100:.1f}% {'✅' if len(winning_trades)/len(trades_df) >= min_win_rate else '❌'} (min: {min_win_rate*100:.0f}%)")
    
    # Plot equity curve
    plt.figure(figsize=(12, 6))
    plt.plot(equity_df['timestamp'], equity_df['equity'])
    plt.title('Portfolio Equity Curve')
    plt.xlabel('Date')
    plt.ylabel('Equity ($)')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Plot drawdown
    plt.figure(figsize=(12, 4))
    plt.fill_between(equity_df['timestamp'], drawdown * 100, 0, alpha=0.3, color='red')
    plt.title('Portfolio Drawdown')
    plt.xlabel('Date')
    plt.ylabel('Drawdown (%)')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("\n⚠️ No trades available for performance analysis")


⚠️ No trades available for performance analysis


## Intraday Pattern Analysis

In [10]:
# Analyze intraday patterns if requested
if analyze_intraday_patterns and len(trades_df) > 0:
    print("\n⏰ INTRADAY PATTERN ANALYSIS")
    print("=" * 80)
    
    # Extract hour of entry and exit
    trades_df['entry_hour'] = trades_df['entry_time'].dt.hour
    trades_df['exit_hour'] = trades_df['exit_time'].dt.hour
    trades_df['entry_day'] = trades_df['entry_time'].dt.dayofweek
    
    # Performance by hour of day
    hourly_performance = trades_df.groupby('entry_hour').agg({
        'pnl': ['count', 'sum', 'mean'],
        'return': 'mean'
    })
    
    # Win rate by hour
    hourly_win_rate = trades_df.groupby('entry_hour').apply(
        lambda x: (x['pnl'] > 0).mean() * 100
    )
    
    # Performance by day of week
    daily_performance = trades_df.groupby('entry_day').agg({
        'pnl': ['count', 'sum', 'mean'],
        'return': 'mean'
    })
    
    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Trades by hour
    ax = axes[0, 0]
    hourly_performance['pnl']['count'].plot(kind='bar', ax=ax)
    ax.set_title('Number of Trades by Hour')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Trade Count')
    
    # Win rate by hour
    ax = axes[0, 1]
    hourly_win_rate.plot(kind='bar', ax=ax, color='green')
    ax.axhline(50, color='red', linestyle='--', alpha=0.5)
    ax.set_title('Win Rate by Hour')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Win Rate (%)')
    
    # Average PnL by hour
    ax = axes[1, 0]
    hourly_performance['pnl']['mean'].plot(kind='bar', ax=ax, color='blue')
    ax.axhline(0, color='red', linestyle='--', alpha=0.5)
    ax.set_title('Average PnL by Hour')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Average PnL ($)')
    
    # Performance by day of week
    ax = axes[1, 1]
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
    daily_performance['pnl']['mean'].plot(kind='bar', ax=ax, color='purple')
    ax.set_xticklabels(days[:len(daily_performance)], rotation=0)
    ax.axhline(0, color='red', linestyle='--', alpha=0.5)
    ax.set_title('Average PnL by Day of Week')
    ax.set_xlabel('Day of Week')
    ax.set_ylabel('Average PnL ($)')
    
    plt.tight_layout()
    plt.show()
    
    # Best and worst times
    print("\n🕐 Best Trading Hours:")
    best_hours = hourly_performance['pnl']['mean'].nlargest(3)
    for hour, avg_pnl in best_hours.items():
        count = hourly_performance.loc[hour, ('pnl', 'count')]
        win_rate = hourly_win_rate.loc[hour]
        print(f"  {hour}:00 - Avg PnL: ${avg_pnl:.2f}, Win Rate: {win_rate:.1f}%, Trades: {count}")
    
    print("\n🕐 Worst Trading Hours:")
    worst_hours = hourly_performance['pnl']['mean'].nsmallest(3)
    for hour, avg_pnl in worst_hours.items():
        count = hourly_performance.loc[hour, ('pnl', 'count')]
        win_rate = hourly_win_rate.loc[hour]
        print(f"  {hour}:00 - Avg PnL: ${avg_pnl:.2f}, Win Rate: {win_rate:.1f}%, Trades: {count}")

## Risk Analysis

In [11]:
# Comprehensive risk analysis
if len(trades_df) > 0:
    print("\n⚠️ RISK ANALYSIS")
    print("=" * 80)
    
    # Trade duration analysis
    print("Trade Duration Statistics:")
    print(f"  Average: {trades_df['duration'].mean():.1f} minutes")
    print(f"  Median: {trades_df['duration'].median():.1f} minutes")
    print(f"  Shortest: {trades_df['duration'].min():.1f} minutes")
    print(f"  Longest: {trades_df['duration'].max():.1f} minutes")
    
    # Consecutive wins/losses
    trades_df['is_win'] = trades_df['pnl'] > 0
    trades_df['streak'] = (trades_df['is_win'] != trades_df['is_win'].shift()).cumsum()
    
    win_streaks = trades_df[trades_df['is_win']].groupby('streak').size()
    loss_streaks = trades_df[~trades_df['is_win']].groupby('streak').size()
    
    print(f"\nStreak Analysis:")
    print(f"  Max consecutive wins: {win_streaks.max() if len(win_streaks) > 0 else 0}")
    print(f"  Max consecutive losses: {loss_streaks.max() if len(loss_streaks) > 0 else 0}")
    print(f"  Average win streak: {win_streaks.mean():.1f}" if len(win_streaks) > 0 else "  Average win streak: N/A")
    print(f"  Average loss streak: {loss_streaks.mean():.1f}" if len(loss_streaks) > 0 else "  Average loss streak: N/A")
    
    # Risk-adjusted returns
    if trades_df['return'].std() > 0:
        information_ratio = trades_df['return'].mean() / trades_df['return'].std()
        print(f"\nRisk-Adjusted Metrics:")
        print(f"  Information Ratio: {information_ratio:.3f}")
        print(f"  Return/Risk: {trades_df['return'].mean() / trades_df['return'].std():.3f}")
    
    # Value at Risk (VaR)
    var_95 = np.percentile(trades_df['pnl'], 5)
    var_99 = np.percentile(trades_df['pnl'], 1)
    
    print(f"\nValue at Risk (VaR):")
    print(f"  95% VaR: ${var_95:.2f}")
    print(f"  99% VaR: ${var_99:.2f}")
    
    # Plot PnL distribution
    plt.figure(figsize=(10, 6))
    plt.hist(trades_df['pnl'], bins=50, alpha=0.7, edgecolor='black')
    plt.axvline(0, color='red', linestyle='--', alpha=0.5, label='Breakeven')
    plt.axvline(trades_df['pnl'].mean(), color='green', linestyle='--', alpha=0.5, label='Mean PnL')
    plt.axvline(var_95, color='orange', linestyle='--', alpha=0.5, label='95% VaR')
    plt.xlabel('PnL ($)')
    plt.ylabel('Frequency')
    plt.title('PnL Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## Summary and Recommendations

In [12]:
# Generate summary and recommendations
print("\n📋 SUMMARY AND RECOMMENDATIONS")
print("=" * 80)

summary = {
    'run_info': {
        'run_id': run_dir.name,
        'config_name': config_name,
        'analysis_timestamp': datetime.now().isoformat(),
        'is_full_system': is_full_system
    },
    'data_summary': {
        'total_bars': metadata.get('total_bars', 0),
        'total_signals': metadata.get('total_signals', 0),
        'total_orders': metadata.get('total_orders', 0),
        'total_fills': metadata.get('total_fills', 0),
        'total_positions': metadata.get('total_positions', 0)
    },
    'performance_summary': {},
    'risk_summary': {},
    'recommendations': []
}

if len(trades_df) > 0:
    # Performance summary
    summary['performance_summary'] = {
        'total_trades': len(trades_df),
        'total_return': float(total_return),
        'sharpe_ratio': float(sharpe_ratio),
        'max_drawdown': float(max_drawdown),
        'win_rate': float(len(winning_trades)/len(trades_df)),
        'profit_factor': float(winning_trades['pnl'].sum() / abs(losing_trades['pnl'].sum())) if len(losing_trades) > 0 and losing_trades['pnl'].sum() != 0 else 0
    }
    
    # Risk summary
    summary['risk_summary'] = {
        'var_95': float(var_95),
        'var_99': float(var_99),
        'max_consecutive_losses': int(loss_streaks.max()) if len(loss_streaks) > 0 else 0,
        'avg_trade_duration_minutes': float(trades_df['duration'].mean())
    }
    
    # Generate recommendations
    if sharpe_ratio < min_sharpe_ratio:
        summary['recommendations'].append({
            'type': 'performance',
            'severity': 'high',
            'message': f'Sharpe ratio ({sharpe_ratio:.2f}) below minimum threshold ({min_sharpe_ratio}). Consider parameter optimization.'
        })
    
    if abs(max_drawdown) > max_acceptable_drawdown:
        summary['recommendations'].append({
            'type': 'risk',
            'severity': 'high',
            'message': f'Maximum drawdown ({abs(max_drawdown)*100:.1f}%) exceeds acceptable limit ({max_acceptable_drawdown*100:.0f}%). Implement stricter risk controls.'
        })
    
    if len(winning_trades)/len(trades_df) < min_win_rate:
        summary['recommendations'].append({
            'type': 'performance',
            'severity': 'medium',
            'message': f'Win rate ({len(winning_trades)/len(trades_df)*100:.1f}%) below minimum ({min_win_rate*100:.0f}%). Review entry criteria.'
        })
    
    # Execution-specific recommendations
    if 'slippage_bps' in fills.columns and fills['slippage_bps'].mean() > 5:
        summary['recommendations'].append({
            'type': 'execution',
            'severity': 'medium',
            'message': f'High average slippage ({fills["slippage_bps"].mean():.1f} bps). Consider limit orders or better execution timing.'
        })
    
    # Intraday pattern recommendations
    if analyze_intraday_patterns and 'hourly_performance' in locals():
        worst_hour = hourly_performance['pnl']['mean'].idxmin()
        if hourly_performance.loc[worst_hour, ('pnl', 'mean')] < -50:
            summary['recommendations'].append({
                'type': 'timing',
                'severity': 'low',
                'message': f'Poor performance at {worst_hour}:00. Consider avoiding trades during this hour.'
            })

# Display recommendations
if summary['recommendations']:
    print("🎯 Recommendations:")
    for rec in sorted(summary['recommendations'], key=lambda x: {'high': 0, 'medium': 1, 'low': 2}[x['severity']]):
        severity_icon = {'high': '🔴', 'medium': '🟡', 'low': '🟢'}[rec['severity']]
        print(f"\n{severity_icon} [{rec['severity'].upper()}] {rec['type'].title()}")
        print(f"   {rec['message']}")
else:
    print("✅ No critical issues identified")

# Save summary
with open(run_dir / 'analysis_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n📄 Analysis summary saved to: analysis_summary.json")


📋 SUMMARY AND RECOMMENDATIONS
✅ No critical issues identified

📄 Analysis summary saved to: analysis_summary.json


## Export Results

In [13]:
# Export key dataframes for further analysis
print("\n💾 EXPORTING RESULTS")
print("=" * 80)

exports = {}

# Export trades if available
if len(trades_df) > 0:
    trades_df.to_csv(run_dir / 'analyzed_trades.csv', index=False)
    exports['trades'] = 'analyzed_trades.csv'
    print(f"✅ Exported {len(trades_df)} trades")

# Export fills analysis if available
if len(fills) > 0 and 'slippage_bps' in fills.columns:
    slippage_summary = fills[['timestamp', 'symbol', 'quantity', 'price', 'expected_price', 'slippage_bps']]
    slippage_summary.to_csv(run_dir / 'slippage_analysis.csv', index=False)
    exports['slippage'] = 'slippage_analysis.csv'
    print(f"✅ Exported slippage analysis for {len(fills)} fills")

# Export performance metrics
if 'equity_df' in locals():
    equity_df.to_csv(run_dir / 'equity_curve.csv', index=False)
    exports['equity_curve'] = 'equity_curve.csv'
    print(f"✅ Exported equity curve")

# Create final report
report = {
    'analysis_complete': True,
    'timestamp': datetime.now().isoformat(),
    'exports': exports,
    'summary': summary
}

with open(run_dir / 'final_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print(f"\n✅ Analysis complete! Results saved to {run_dir}")


💾 EXPORTING RESULTS

✅ Analysis complete! Results saved to /Users/daws/ADMF-PC/config/bollinger/results/20250703_201201
